## Hyperparameter Search for Deep Learning model

##### This notebook illustrates how our deep learning model is trained. Optimal network parameters are determined via a hyperparameter search.

Import required libraries:

In [ ]:
import optunity
import lifelines
from lifelines.utils import concordance_index
import os, sys, pickle
import numpy as np

We import ```DL_single_run``` (which performs one run of the DL model) from the ```trainDL.py``` file.

In [ ]:
sys.path.insert(0, '../code')
from trainDL import *

Import simulated data. This serialized ```.pkl``` file consists of the following variables: 
- ```x_full``` : simulated input mesh motion data (i.e. motion descriptors [11,514-element vector])
- ```y_full``` : simulated censoring status and survival times


In [ ]:
with open('../data/inputdata_DL.pkl', 'rb') as f: c3 = pickle.load(f)
x_full = c3[0]
y_full = c3[1]
del c3

The ```hypersearch_DL()``` function is defined in the code block below. It searches for optimal hyperparameters for our deep learning model. The function takes as arguments the input and outcome data variables, hyperparameter search method, search parameters (number iterations, number of cross-validation folds), and search boundaries for each hyperparameter.

The function evaluates various hyperparameter sets by using each to fit our deep learning model (see the ```modelrun()``` function below). For a stable estimate of performance, each hyperparameter set is evaluated over cross-validation folds, with the average performance reported. Here, performance is assessed by _Harrell's_ Concordance Index (see ```concordance_index``` function below). Cross-validation is carried out using the ```optunity.cross_validated``` function decorator, applied to the ```modelrun()``` function. The hyperparameter set yielding the best cross-validated performance is reported and returned (see the last 3 lines of the function definition below).

In [ ]:
def hypersearch_DL(x_data, y_data, method, nfolds, nevals, lrexp_range, l1rexp_range, dro_range, 
                   units1_range, units2_range, alpha_range, batch_size, num_epochs):

    @optunity.cross_validated(x=x_data, y=y_data, num_folds=nfolds)
    def modelrun(x_train, y_train, x_test, y_test, lrexp, l1rexp, dro, units1, units2, alpha):
        cv_log = DL_single_run(xtr=x_train, ytr=y_train, units1=units1, units2=units2, dro=dro, lr=10**lrexp, 
                               l1r=10**l1rexp, alpha=alpha, batchsize=batch_size, numepochs=num_epochs)
        cv_preds = cv_log.model.predict(x_test, batch_size=1)[1]
        cv_C = concordance_index(y_test[:,1], -cv_preds, y_test[:,0])
        return cv_C

    optimal_pars, searchlog, _ = optunity.maximize(modelrun, num_evals=nevals, solver_name=method, lrexp=lrexp_range, 
                                                   l1rexp=l1rexp_range, dro=dro_range, units1=units1_range, 
                                                   units2=units2_range, alpha=alpha_range)

    print('Optimal hyperparameters : ' + str(optimal_pars))
    print('Cross-validated C after tuning: %1.3f' % searchlog.optimum)
    return optimal_pars, searchlog

Below, we provide a detailed description of each argument of the ```hypersearch_DL``` function:
- **x_data** : input data (mesh motion descriptors)
- **y_data** : outcome data (censoring status and survival times)
- **method** : Hyperparameter search technique. Below are examples of possible options (for full list of search techniques, see [Optunity solvers](https://optunity.readthedocs.io/en/latest/user/solvers.html) ):
  - '_grid search_' : sample the hyperparameter space using a grid search
  - '_random_' : sample the hyperparameter space using a random search
  - '_sobol_' : sample the hyperparameter space using a [Sobol sequence](https://optunity.readthedocs.io/en/latest/user/solvers/sobol.html#sobol)
  - '_particle swarm_' : search hyperparameter space using [Particle Swarm Optimization](https://optunity.readthedocs.io/en/latest/user/solvers/particle_swarm.html#pso2010)
- **nfolds** : number of cross-validation folds
- **nevals** : number of hyperparameter search evaluations
- **lrexp_range** : range of learning rate values, expressed in log<sub>10</sub> base
- **l1rexp_range** : range of _L_<sub>1</sub> regularization values, expressed in log<sub>10</sub> base
- **dro_range** : range of dropout values
- **units1_range** : range of number of units for autoencoder hidden layers, specifically the layer immediately after the encoder's random noise layer, and the one immediately preceding the decoder's output layer
- **units2_range** : range of number of units for the latent code layer ('central' layer of the autoencoder)
- **alpha_range** : range of values for parameter &alpha;, which controls the tradeoff between the 2 losses used in our network (i.e. reconstruction and prediction losses) 
- **batch_size** : batch size
- **num_epochs** : number of epochs


In [ ]:
opars, clog = hypersearch_DL(x_data=x_full, y_data=y_full, 
                             method='particle swarm', nfolds=6, nevals=50,
                             lrexp_range=[-6.,-4.5], l1rexp_range=[-7,-4], dro_range=[.1,.9], 
                             units1_range=[75,250],  units2_range=[5,20],  alpha_range=[0.3, 0.7],
                             batch_size=16, num_epochs=100)